In [125]:
import torch
from torch import nn, optim
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import csv

In [126]:
device = torch.device("cuda")
data = pd.read_csv('train_data.csv')
data_dropped = data.drop(columns=['HallwayType','HeatingType','AptManageType','SubwayStation', 'TimeToBusStop', 'TimeToSubway'])
pd_HallwayTypepd = pd.get_dummies(data['HallwayType'],  dtype=float)
pd_HeatingType = pd.get_dummies(data['HeatingType'],  dtype=float)
pd_AptManageType = pd.get_dummies(data['AptManageType'],  dtype=float)
pd_SubwayStation = pd.get_dummies(data['SubwayStation'],  dtype=float)
pd_TimeToSubway = pd.get_dummies(data['TimeToSubway'],  dtype=float)
pd_TimeToBusStop = pd.get_dummies(data['TimeToBusStop'],  dtype=float)
data_encoded = pd.concat([pd_HallwayTypepd,pd_HeatingType ,pd_AptManageType,pd_SubwayStation,pd_TimeToSubway,pd_TimeToBusStop , data_dropped], axis=1, join='outer')
X = data_encoded.drop(columns=['SalePrice']).values
Y = data_encoded['SalePrice'].values
print(X)
print(Y)
print(X.shape)
print(Y.shape)

[[ 0.  0.  1. ...  5.  6.  9.]
 [ 1.  0.  0. ...  3. 12.  4.]
 [ 1.  0.  0. ...  3. 12.  4.]
 ...
 [ 0.  0.  1. ... 10.  9. 10.]
 [ 1.  0.  0. ...  3.  7. 11.]
 [ 0.  0.  1. ... 10.  9. 10.]]
[141592  51327  48672 ... 357522 312389 393805]
(4124, 33)
(4124,)


In [127]:
test_data = pd.read_csv('test_data.csv')
test_data_dropped = test_data.drop(columns=['HallwayType','HeatingType','AptManageType','SubwayStation', 'TimeToBusStop', 'TimeToSubway'])
pd_HallwayTypepd = pd.get_dummies(test_data['HallwayType'],  dtype=float)
pd_HeatingType = pd.get_dummies(test_data['HeatingType'],  dtype=float)
pd_AptManageType = pd.get_dummies(test_data['AptManageType'],  dtype=float)
pd_SubwayStation = pd.get_dummies(test_data['SubwayStation'],  dtype=float)
pd_TimeToSubway = pd.get_dummies(test_data['TimeToSubway'],  dtype=float)
pd_TimeToBusStop = pd.get_dummies(test_data['TimeToBusStop'],  dtype=float)
test_data_encoded = pd.concat([pd_HallwayTypepd,pd_HeatingType ,pd_AptManageType,pd_SubwayStation,pd_TimeToSubway,pd_TimeToBusStop , test_data_dropped], axis=1, join='outer')

test_data = test_data_encoded.astype('float32')
test_data = torch.from_numpy(test_data.values[:,:])
print(test_data)

tensor([[ 0.,  1.,  0.,  ...,  4., 14., 17.],
        [ 0.,  0.,  1.,  ...,  9., 14., 17.],
        [ 0.,  0.,  1.,  ...,  5.,  9.,  5.],
        ...,
        [ 0.,  1.,  0.,  ...,  4., 14., 17.],
        [ 0.,  0.,  1.,  ...,  8.,  7.,  9.],
        [ 0.,  1.,  0.,  ...,  4., 14., 17.]])


In [128]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

tensor_list = []
for price in y_train:
    if price <= 100000:
        tensor_list.append(torch.tensor([1.0, 0.0, 0.0]))
    elif price > 350000:
        tensor_list.append(torch.tensor([0.0, 0.0, 1.0]))
    else:
        tensor_list.append(torch.tensor([0.0, 1.0, 0.0]))

y_train = torch.stack(tensor_list)

train_data = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)


In [129]:
class PricePredictionModel(nn.Module):

    def __init__(self, num_inputs):
        super().__init__()
        self.linear1 = nn.Linear(num_inputs, 64)
        self.act_fn = nn.ReLU()
        self.linear2 = nn.Linear(64, 64)
        self.linear3 = nn.Linear(64, 128)
        # self.linear4 = nn.Linear(128, 256)
        # self.linear5 = nn.Linear(256, 128)
        self.linear6 = nn.Linear(128, 64)
        self.linear7 = nn.Linear(64, 3)

    def forward(self, x):
        print(x.shape)
        x = self.linear1(x)
        x = self.act_fn(x)
        x = self.linear2(x)
        x = self.act_fn(x)
        x = self.linear3(x)
        x = self.act_fn(x)
        # x = self.linear4(x)
        # x = self.act_fn(x)
        # x = self.linear5(x)
        # x = self.act_fn(x)
        x = self.linear6(x)
        x = self.act_fn(x)
        x = self.linear7(x)
        return x


In [130]:
# Inicjalizacja modelu, funkcji straty i optymizatora
num_inputs = 33
model = PricePredictionModel(num_inputs)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [131]:
# Trening modelu
epochs = 100
for epoch in range(epochs):
    # optimizer.zero_grad()
    # outputs = model(X_train)
    # loss = criterion(outputs, y_train)
    # loss.backward()
    # optimizer.step()
    # if (epoch+1) % 10 == 0:
    #     print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')
    for inputs, labels in train_loader:
            # inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels.view(-1, 1))

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')




torch.Size([64, 33])


ValueError: Expected input batch_size (64) to match target batch_size (192).

In [ ]:
# Ocena modelu na zbiorze testowym
with torch.no_grad():
    outputs = model(X_test)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == y_test).sum().item() / len(y_test)
    print(f'Accuracy on test set: {accuracy:.2f}')

Accuracy on test set: 0.00


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Załaduj zestaw danych Iris
iris = load_iris()
X = iris.data
y = iris.target

# Podział danych na zbiór treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standaryzacja danych
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Konwersja danych na tensory
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

# Definicja modelu
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(4, 8)  # Warstwa wejściowa
        self.fc2 = nn.Linear(8, 3)  # Warstwa wyjściowa

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Inicjalizacja modelu, funkcji straty i optymizatora
model = SimpleNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Trening modelu
epochs = 100
for epoch in range(epochs):
    # optimizer.zero_grad()
    # outputs = model(X_train)
    # loss = criterion(outputs, y_train)
    # loss.backward()
    # optimizer.step()
    # if (epoch+1) % 10 == 0:
    #     print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

    

# Ocena modelu na zbiorze testowym
with torch.no_grad():
    outputs = model(X_test)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == y_test).sum().item() / len(y_test)
    print(f'Accuracy on test set: {accuracy:.2f}')


RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x33 and 4x8)